# Yelp Adding IRS Income

We want to see if Yelp business price ratings ($) correlate with the affluence of a given ZIP code. We'll compare to both median gross income and mean gross income. This data was collected from [https://factfinder.census.gov/](https://factfinder.census.gov/).

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
df = pd.read_csv('../data/master.csv')
df.head()

,alias,category,city,id,latitude,longitude,price,rating,review_count,zip_code
0,lucilles-smokehouse-bar-b-que-brea,"['bbq', 'smokehouse', 'southern']",Brea,zzvkb4J_sQAk8N20OWdZhg,33.911154,-117.881259,2,3.5,1257.0,92821.0
1,gen-korean-bbq-house-northridge-2,"['korean', 'bbq', 'asianfusion']",Northridge,zzcBtsGd7uFQ7AwIu_GcQQ,34.255146,-118.537373,2,4.5,3533.0,91324.0
2,mariscos-camarena-estilo-nayarit-san-fernando,"['seafood', 'bars']",San Fernando,zzLBuLz4ORQF0V_QwgHtBA,34.299215,-118.438575,2,2.5,91.0,91340.0
3,sub-king-signal-hill,"['sandwiches', 'breakfast_brunch']",Signal Hill,zzJJxXDkai8rR2tBX0Ggrw,33.804625,-118.165310,1,4.5,76.0,90755.0
4,bardonna-los-angeles-3,"['coffee', 'breakfast_brunch', 'cafes']",Los Angeles,zz0zctGWZ4vkxyx00LOIKw,34.074060,-118.323819,2,3.5,174.0,90004.0


In [29]:
df.shape

(27427, 10)

We'll extract the ZIP codes in our dataframe and save to a variable for easy reference.

In [30]:
zip_list = list(df['zip_code'].sort_values().unique())
zip_list = [int(z) for z in zip_list]
zip_list

[90001,
 90002,
 90003,
 90004,
 90005,
 90006,
 90007,
 90008,
 90009,
 90010,
 90011,
 90012,
 90013,
 90014,
 90015,
 90016,
 90017,
 90018,
 90019,
 90020,
 90021,
 90022,
 90023,
 90024,
 90025,
 90026,
 90027,
 90028,
 90029,
 90031,
 90032,
 90033,
 90034,
 90035,
 90036,
 90037,
 90038,
 90039,
 90040,
 90041,
 90042,
 90043,
 90044,
 90045,
 90046,
 90047,
 90048,
 90049,
 90052,
 90056,
 90057,
 90058,
 90059,
 90060,
 90061,
 90062,
 90063,
 90064,
 90065,
 90066,
 90067,
 90068,
 90069,
 90071,
 90073,
 90077,
 90078,
 90079,
 90089,
 90094,
 90095,
 90101,
 90189,
 90201,
 90209,
 90210,
 90211,
 90212,
 90220,
 90221,
 90222,
 90230,
 90232,
 90240,
 90241,
 90242,
 90245,
 90247,
 90248,
 90249,
 90250,
 90254,
 90255,
 90260,
 90261,
 90262,
 90263,
 90265,
 90266,
 90270,
 90272,
 90274,
 90275,
 90277,
 90278,
 90280,
 90290,
 90291,
 90292,
 90293,
 90301,
 90302,
 90303,
 90304,
 90305,
 90312,
 90401,
 90402,
 90403,
 90404,
 90405,
 90407,
 90421,
 90501,
 90502,


## Import IRS `.csv`

In [31]:
acs = pd.read_csv('../Assets/2017irs.csv', header=1)
acs.head(10)

,Id,Id2,Geography,Households; Estimate; Total,Households; Margin of Error; Total,Families; Estimate; Total,Families; Margin of Error; Total,Married-couple families; Estimate; Total,Married-couple families; Margin of Error; Total,Nonfamily households; Estimate; Total,...,Nonfamily households; Estimate; PERCENT ALLOCATED - Family income in the past 12 months,Nonfamily households; Margin of Error; PERCENT ALLOCATED - Family income in the past 12 months,Households; Estimate; PERCENT ALLOCATED - Nonfamily income in the past 12 months,Households; Margin of Error; PERCENT ALLOCATED - Nonfamily income in the past 12 months,Families; Estimate; PERCENT ALLOCATED - Nonfamily income in the past 12 months,Families; Margin of Error; PERCENT ALLOCATED - Nonfamily income in the past 12 months,Married-couple families; Estimate; PERCENT ALLOCATED - Nonfamily income in the past 12 months,Married-couple families; Margin of Error; PERCENT ALLOCATED - Nonfamily income in the past 12 months,Nonfamily households; Estimate; PERCENT ALLOCATED - Nonfamily income in the past 12 months,Nonfamily households; Margin of Error; PERCENT ALLOCATED - Nonfamily income in the past 12 months
0,8600000US89010,89010,ZCTA5 89010,208,54,122,33,109,31,86,...,(X),(X),(X),(X),(X),(X),(X),(X),43.0,(X)
1,8600000US89019,89019,ZCTA5 89019,788,203,354,143,211,105,434,...,(X),(X),(X),(X),(X),(X),(X),(X),28.8,(X)
2,8600000US89046,89046,ZCTA5 89046,212,56,85,35,58,29,127,...,(X),(X),(X),(X),(X),(X),(X),(X),64.6,(X)
3,8600000US89060,89060,ZCTA5 89060,3874,279,2245,302,1838,238,1629,...,(X),(X),(X),(X),(X),(X),(X),(X),37.0,(X)
4,8600000US89061,89061,ZCTA5 89061,2332,231,1588,227,1382,227,744,...,(X),(X),(X),(X),(X),(X),(X),(X),27.3,(X)
5,8600000US89439,89439,ZCTA5 89439,667,84,446,81,365,88,221,...,(X),(X),(X),(X),(X),(X),(X),(X),35.3,(X)
6,8600000US90001,90001,ZCTA5 90001,13807,276,11345,270,6126,329,2462,...,(X),(X),(X),(X),(X),(X),(X),(X),33.7,(X)
7,8600000US90002,90002,ZCTA5 90002,12694,271,10294,291,5009,334,2400,...,(X),(X),(X),(X),(X),(X),(X),(X),41.0,(X)
8,8600000US90003,90003,ZCTA5 90003,16581,368,13378,411,6418,349,3203,...,(X),(X),(X),(X),(X),(X),(X),(X),32.7,(X)
9,8600000US90004,90004,ZCTA5 90004,22140,370,12942,456,8156,435,9198,...,(X),(X),(X),(X),(X),(X),(X),(X),30.1,(X)


In [32]:
acs.columns = [col.lower() for col in acs.columns.str.replace(' ', '_').str.replace(';', '_')]
acs.head()

,id,id2,geography,households__estimate__total,households__margin_of_error__total,families__estimate__total,families__margin_of_error__total,married-couple_families__estimate__total,married-couple_families__margin_of_error__total,nonfamily_households__estimate__total,...,nonfamily_households__estimate__percent_allocated_-_family_income_in_the_past_12_months,nonfamily_households__margin_of_error__percent_allocated_-_family_income_in_the_past_12_months,households__estimate__percent_allocated_-_nonfamily_income_in_the_past_12_months,households__margin_of_error__percent_allocated_-_nonfamily_income_in_the_past_12_months,families__estimate__percent_allocated_-_nonfamily_income_in_the_past_12_months,families__margin_of_error__percent_allocated_-_nonfamily_income_in_the_past_12_months,married-couple_families__estimate__percent_allocated_-_nonfamily_income_in_the_past_12_months,married-couple_families__margin_of_error__percent_allocated_-_nonfamily_income_in_the_past_12_months,nonfamily_households__estimate__percent_allocated_-_nonfamily_income_in_the_past_12_months,nonfamily_households__margin_of_error__percent_allocated_-_nonfamily_income_in_the_past_12_months
0,8600000US89010,89010,ZCTA5 89010,208,54,122,33,109,31,86,...,(X),(X),(X),(X),(X),(X),(X),(X),43.0,(X)
1,8600000US89019,89019,ZCTA5 89019,788,203,354,143,211,105,434,...,(X),(X),(X),(X),(X),(X),(X),(X),28.8,(X)
2,8600000US89046,89046,ZCTA5 89046,212,56,85,35,58,29,127,...,(X),(X),(X),(X),(X),(X),(X),(X),64.6,(X)
3,8600000US89060,89060,ZCTA5 89060,3874,279,2245,302,1838,238,1629,...,(X),(X),(X),(X),(X),(X),(X),(X),37.0,(X)
4,8600000US89061,89061,ZCTA5 89061,2332,231,1588,227,1382,227,744,...,(X),(X),(X),(X),(X),(X),(X),(X),27.3,(X)


## Extract median / mean household income

In [33]:
acs.rename(columns={'id2': 'zip_code',
                    'households__estimate__median_income_(dollars)': 'med_agi',
                    'households__estimate__mean_income_(dollars)': 'avg_agi'}, inplace=True)
columns = ['zip_code', 
           'med_agi', 
           'avg_agi']
acs[columns]

,zip_code,med_agi,avg_agi
0,89010,46111,56505
1,89019,35856,55862
2,89046,31389,48033
3,89060,38370,47664
4,89061,54375,65354
...,...,...,...
1770,97603,46125,58900
1771,97623,42448,53052
1772,97632,40536,53140
1773,97633,42611,57167


In [34]:
acs[columns].dtypes

zip_code     int64
med_agi     object
avg_agi     object
dtype: object

## Only use ZIP codes that are present in our DataFrame

In [35]:
zip_agi = acs.loc[acs['zip_code'].isin(zip_list), columns]
zip_agi.head()

,zip_code,med_agi,avg_agi
6,90001,35660,45650
7,90002,34000,44463
8,90003,34397,43342
9,90004,46581,82091
10,90005,32461,49310


## Combine income data with our DataFrame

In [36]:
df_income = pd.merge(df, zip_agi, how = 'outer' )
df_income

,alias,category,city,id,latitude,longitude,price,rating,review_count,zip_code,med_agi,avg_agi
0,lucilles-smokehouse-bar-b-que-brea,"['bbq', 'smokehouse', 'southern']",Brea,zzvkb4J_sQAk8N20OWdZhg,33.911154,-117.881259,2,3.5,1257.0,92821.0,86070,106163
1,subway-restaurants-brea-4,['sandwiches'],Brea,zWs68lBY2PyNOTRJhxXapg,33.911959,-117.867374,1,3.0,22.0,92821.0,86070,106163
2,nekter-juice-bar-brea,"['juicebars', 'acaibowls', 'icecream']",Brea,zOYNwoDjpg6Ehwn0RKX9qA,33.911283,-117.867449,2,3.5,367.0,92821.0,86070,106163
3,arbys-brea-2,"['sandwiches', 'hotdogs']",Brea,zNNLl821AlzApHYqRxNALw,33.911431,-117.865941,1,2.5,84.0,92821.0,86070,106163
4,portofino-trattoria-brea,['italian'],Brea,zJ4pvxtpo3Y5LvylLXmCzA,33.932860,-117.916253,2,4.0,231.0,92821.0,86070,106163
...,...,...,...,...,...,...,...,...,...,...,...,...
27422,northridge-hospital-cafeteria-northridge,['tradamerican'],Northridge,3OnkDJAW9MMef5VkbFWY8w,34.219970,-118.532930,1,4.5,7.0,91328.0,NaN,NaN
27423,sbarro-brea,"['italian', 'pizza']",Brea,2xrnvWPB0VTcSxMDL4Dqzw,33.916132,-117.885852,1,2.5,43.0,92621.0,NaN,NaN
27424,jojo-cakes-santa-ana-3,"['bakeries', 'desserts', 'cupcakes']",Santa Ana,2V4BH14_2hLxRshiYKdpmQ,33.717411,-117.914160,2,4.5,59.0,92704.0,63056,77776
27425,cosmic-cafe-mt-wilson,['cafes'],Mt Wilson,1RshW3QfxW0gtaXajWZyQw,34.223577,-118.061963,1,4.0,29.0,91023.0,NaN,NaN


## Deal with erroneous (or new) ZIP codes

Some ZIP codes do not have income data from 2017. We suspect these ZIP codes are either new or erroneously entered into the Yelp website.

In [37]:
df_income.dropna(inplace = True)
df_income.shape

(27311, 12)

In [38]:
dash = list(df_income[df_income['med_agi'].str.contains('-')].index)
df_income.drop(index=dash, inplace=True)
df_income.shape

(27119, 12)

In [39]:
df_income.dtypes

alias            object
category         object
city             object
id               object
latitude        float64
longitude       float64
price             int64
rating          float64
review_count    float64
zip_code        float64
med_agi          object
avg_agi          object
dtype: object

In [40]:
df_income['avg_agi'] = df_income['avg_agi'].astype(int)
df_income['med_agi'] = df_income['med_agi'].astype(int)
df_income['zip_code'] = df_income['zip_code'].astype(str)

In [41]:
df_income.dtypes

alias            object
category         object
city             object
id               object
latitude        float64
longitude       float64
price             int64
rating          float64
review_count    float64
zip_code         object
med_agi           int64
avg_agi           int64
dtype: object

We'll form a `count` column for later.

In [42]:
df_income['count'] = df_income.groupby('zip_code')['zip_code'].transform('count')

## Save the combined data to a `.csv`

In [44]:
df_income.to_csv('../data/data_income.csv', index=False)

## Combined data organized by zipcode for Flask App

In [45]:
df_income['zip_code'] = df_income['zip_code'].astype(str)
df_zipcode = df_income.pivot_table(index = ['zip_code']).reset_index()

In [46]:
df_zipcode.to_csv('../data/data_zipcode.csv')

# Move on to 04 - Yelp Model Selection